In [1]:
import csv as csv 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
# %matplotlib outline 

# Open up the csv file in to a Python object
data = pd.read_csv('data.csv', header=0)
data = data.reindex(np.random.permutation(data.index))


# dataFrame.dtypes
# dataFrame.info()
# dataFrame.describe()
# dataFrame.x.mean()

# dataFrame['x'].hist(bins=16, range=(-5,5), alpha = .2)
#P.show()
# dataFrame.head()
# type(dataFrame.x)



/home/hd/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:


X_train1 = data.iloc[0:80,0]
Y_train = data.iloc[0:80,1]
Y_train1 = Y_train

X_test1 = data.iloc[80:100,0]
Y_test = data.iloc[80:100,1]


x_plot = np.linspace(-5, 5, 100)
# create matrix versions of these arrays
X_train = X_train1[:, np.newaxis]
X_plot = x_plot[:, np.newaxis]
X_test = X_test1[:,np.newaxis]




In [3]:


low = 3
high = 9
deg = [5,7,9,11]
error = []

plt.scatter(X_train, Y_train, label="training points")
for degree in deg:
    model = make_pipeline(PolynomialFeatures(degree), Ridge())
    model.fit(X_train, Y_train)
    y_plot = model.predict(X_plot)
    
    #calculate the error on test data
    Y_predicted = model.predict(X_test)
    err = np.sum((Y_predicted - Y_test)**2)/20
    error.append(err)
    
#     print(model.get_params)
    
    plt.plot(x_plot, y_plot, label="degree %d" % degree)

plt.legend(loc='lower left')

plt.show()



In [4]:
#Error vs degree
b = deg
plt.scatter(b, error)
plt.plot(b, error, label="Error vs degree")
plt.legend(loc='lower left')
plt.show()



In [5]:
#tune the parameter alpha
#degree = 5
alfas = [0,0.1,1,5,10,20,30,100,10000,1000000000000,100000000000000000]
error = []
for alfa in alfas:
    model = make_pipeline(PolynomialFeatures(5), Ridge(alfa))
    model.fit(X_train, Y_train1)
    y_plot = model.predict(X_plot)
    
    #calculate the error on test data
    Y_predicted = model.predict(X_test)
    err = np.sum((Y_predicted - Y_test)**2)/20
    error.append(err)
    
    
    plt.plot(x_plot, y_plot, label="alpha %d" % alfa)

plt.legend(loc='lower left')

plt.show()

In [6]:
#Error vs alpha
import math
b = alfas
plt.scatter(b, (error))
plt.plot(b, error, label="Error vs alpha")
plt.legend(loc='lower right')
plt.show()

In [7]:
#Finding Variance
#use degree = 5, alpha = 10
from sklearn.linear_model import Lasso
data = pd.read_csv('data.csv', header=0)
X,Y = data.x, data.y
X = X[:, np.newaxis]
model = make_pipeline(PolynomialFeatures(7), Lasso(1000))
model.fit(X, Y)
Y_predicted = model.predict(X)

G = (Y_predicted - Y)**2
err = np.sum(G)/100

print("variance = ",err)




('variance = ', 64956.12673528609)


In [ ]:
#part 1, no regularization

z = np.polyfit(X_train, Y_train,3)
print(z)
hypothesis = np.poly1d(z)

y_new1 = hypothesis(X_train)
 

x_new = np.arange(-5.1, -3.0, 0.1)
y_new = hypothesis(x_new)

plt.plot(X_train,Y_train,'o', x_new, y_new)
plt.show()

error = 0;
for i in range(0,20): 
        error += ((y_new1 - Y_train)**2).sum()

print((error/20)**0.5)

In [ ]:
#code for univariate regression

X,Y = data.x, data.y
#no. of observations
m = Y.size



def h(theta,x):
    '''theta is a column vector, x is a real no
    returns (theta.T)(X)'''
    result = 0
    for i in range(theta.size):
            result += theta[i]*(x**i)
    return result        
            


def cost(y,x,h,theta,lamda):
    #y is the target data, x is the input data, h is the hypothesis, theta is the parameter vector
    m = y.size
    error = 0;
    predictions = h(theta,X)
    #print(predictions)
    sqErrors = (predictions - y) ** 2
    J =  sqErrors.sum() + lamda*np.square(theta).sum()
    J /= m
    return J**(0.5)

def optimize(w):
    return cost(Y,X,h,w,lamda)


#theta[i] = coeff of X**i
#theta is the initial guess
degree = 5;
theta = np.ones(shape=(1+degree,1))
#set the regularization parameter
lamda =  10

print(cost(Y,X,h,theta,lamda))
theta_new = minimize(optimize,theta)
print(cost(Y,X,h,theta_new.x,lamda))

Y_predicted = h(theta_new.x,X)

# plt.plot(X, Y_predicted,color="red",label="Predicted")
# plt.plot(X,Y,color="blue",label="Original")
# plt.legend(loc='upper left')
# plt.grid(True)
# plt.show()
print(theta_new.x)




In [ ]:
sigma = (((Y_predicted-Y)**2).sum())/100
print(sigma)

In [ ]:
costArray = np.zeros(shape=(10,10))
def regress(deg, l):
    
    lamda = l
    theta_init = np.ones(shape=(1+deg,1))
    theta_new = minimize(optimize,theta_init)
    return cost(Y_,X,h,theta_new.x,lamda)

for i in range(1,11):
    for j in range(0,10):
        costArray[i-1,j] = regress(i,j*3)

In [ ]:
print(costArray)

# from mpl_toolkits.mplot3d import Axes3D
# Xaxis = range(0,10)


# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.scatter(Xaxis, Yaxis, costArray[:,0], c='r', marker='o')
# ax.set_xlabel('X Label')
# ax.set_ylabel('Y Label')
# ax.set_zlabel('Z Label')

# b = costArray[5,:]
# print(b)

# plt.plot(Xaxis,b)
# plt.show()

np.where(costArray == costArray.min())
